Preprocessing Chicken Data
1. same size
2. filter the data (sick, dead, healthy)
   

In [2]:
# %pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [8]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [11]:
# %pip install roboflow
from roboflow import Roboflow
import os 
from dotenv import load_dotenv
load_dotenv()

roboAPI = os.getenv("ROBO_API_KEY")
rf = Roboflow(roboAPI)
project = rf.workspace("john-adams-lz9ni").project("dead-healthy-chiken-75zup")
version = project.version(1)
dataset = version.download("tfrecord")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dead-,healthy-chiken-1 in tfrecord:: 100%|██████████| 10/10 [00:00<00:00, 567.37it/s]


In [ ]:
import tensorflow as tf

# Define feature description based on the TFRecord format
feature_description = {
    "image/encoded": tf.io.FixedLenFeature([], tf.string),  # Encoded image
    "image/class/label": tf.io.FixedLenFeature([], tf.int64),  # Class label
}

# Function to parse each TFRecord example
def _parse_function(proto):
    example = tf.io.parse_single_example(proto, feature_description)
    
    # Decode image
    image = tf.io.decode_jpeg(example["image/encoded"], channels=3)
    image = tf.image.resize(image, (224, 224))  # Resize to match CNN input size
    image = image / 255.0  # Normalize pixel values to [0, 1]
    
    label = example["image/class/label"]  # Extract the label
    return image, label

# Function to load the TFRecord dataset
def load_tfrecord_dataset(tfrecord_path, batch_size=32):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = raw_dataset.map(_parse_function)  # Parse the data
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load train and validation datasets
train_dataset = load_tfrecord_dataset('../data/dataset/train/chikens-S5Lb.tfrecord')
valid_dataset = load_tfrecord_dataset('../data/dataset/valid/chikens-S5Lb.tfrecord')


2025-03-01 18:04:33.177528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from tensorflow.keras import layers, models

# Build a simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')  # Output layer for 2 classes: Healthy, Dead
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # For integer labels
              metrics=['accuracy'])

# Summary of the model
model.summary()


Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,218 (42.61 MB)

 Trainable params: 11,169,218 (42.61 MB)

 Non-trainable params: 0 (0.00 B)